## Álgebra Lineal Computacional - 2C 2022


### Sistemas lineales - Métodos iterativos

In [ ]:
import numpy as np

#### Ejercicio 1
Sea $A\in\mathbb{R}^{3\times3}$ , $A=\left(\begin{matrix} 3 & 1 & 1 \\ -2 & 4 & 1 \\ 1 & 1 & -3\end{matrix}\right)$ y sea $b=\left(\begin{matrix} 5 \\ 3 \\ -1 \end{matrix}\right)$.

#### a) 
Probar que los métodos de Jacobi y Gauss-Seidel son convergentes.
#### b) 
Para el **ejercicio 1** de la **P5** debemos implementar los métodos de Jacobi y Gauss-Seidel de forma matricial, usando
$$A= L + D + U$$
Ingresemos la matriz $A$ e investiguemos los comandos `np.triu`, `np.diag` y `np.tril` como sugiere el ejercicio.

In [ ]:
A = np.array([[3,1,1],[-2,4,1],[1,1,-3]])
b = np.array([5,3,-1])
print(np.triu(A))
print(np.triu(A,1))#esta es la que sirve para U
print(np.tril(A))
print(np.tril(A,-1))#esta es la que sirve para L
print(np.diag(A))
print(np.diag(np.diag(A))) #esta es la que sirve para D 

#### c)
Ahora sí resolvamos el **Ej1 P5**: 
Escribir un programa que implemente el método de Jacobi en forma matricial y otro que implemente el método de Gauss-Seidel para la resolución de un sistema lineal $Ax = b$ con las siguientes condiciones:
- que finalice si el método se estaciona,
- que finalice con una advertencia si se excede cierto tope de iteraciones.

Recordar que la matriz de Jacobi es $T=-D^{-1}(L+U)$
y la matriz de Gauss-Seidel es $T=-(L+D)^{-1} U$.

In [ ]:
def Jacobi(A,b,tol=1e-10,m=100,x0=[]): # Elegimos una tolerancia para nuestro error y un máximo de 100 iteraciones
    n    = A.shape[0]
    d    = np.diag(A)
    invD = np.diag(1/d)     # Es la inversa de D
    N    = np.tril(A,-1)+np.triu(A,1)
    T   = -invD@N           # Matriz de iteraciones de Jacobi
    c    = invD@b
    if len(x0) == 0:        # Si no se ingresa un vector inicial, le damos un vector inicial aleatorio  
        x = np.random.random(n)
    else:
        x = x0

    xold = np.zeros(n)       
    i=0
    while np.linalg.norm(x-xold)>tol and i<m: 
        xold=x.copy()      #### SIEMPRE USAR COPY PARA VECTORES Y MATRICES
        x = T@x + c
        i=i+1
        if i==m:
            print('ATENCIÓN: se alcanzó el número máximo de iteraciones')
    return x,i-1           ### pido el vector solución y la cantidad de iteraciones realizadas

In [ ]:
Jacobi(A,b) #probemos

Ahora implementen el de Gauss-Seidel. Para calcular $M^{-1}$ en vez de usar ``np.linalg.inv(M)`` se puede usar ``np.linalg.solve(M,np.eye())``

In [ ]:
def Gauss_Seidel (A,b,tol=1e-10,m=100,x0=[]):
    ###########COMPLETAR##########   


#### Ejercicio 2
#### a) 
Escribir un programa que reciba una matriz $A$, un vector $b$, y realice el método de Jacobi para resolver el sistema $Ax = b$ utilizando las ecuaciones
$$x_i^{(k+1)}=\frac{b_i-\sum_{j \neq i} a_{ij} x_j^{(k)}}{a_{ii}},\quad i=1,2,\dots ,n.$$

Verificar el programa con el ejemplo anterior.
#### b)
Escribir un programa que reciba una matriz $A$, un vector $b$, y realice el método de Gauss-Seidel para resolver el sistema $Ax = b$ utilizando las ecuaciones
$$
{\displaystyle x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum _{j=i+1}^{n}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\dots ,n.}
$$

In [ ]:
def Jacobi_ec(A,b,x0=[],tol=1e-10,m=100):
    n = A.shape[0]
    if len(x0) == 0:
        x = np.random.random(n)
    else:
        x = x0

    xOld = np.zeros(n)
    s=0               #Le cambio el nombre al contador para no confundirnos los índices
    
    while np.linalg.norm(x-xOld)>tol and s<=m:    
        xOld = x.copy()
        for i in range(n):
            x[i] = (b[i] - (A[i,:]@xOld - A[i,i] * xOld[i])) / A[i,i] #Observar que sacamos el término ii de la sumatoria
        s=s+1
    if s==m:
        print('ATENCIÓN: se alcanzó el número máximo de iteraciones')
       
    return x,s-1

In [ ]:
def Gauss_Seidel_ec(A,b,x0=[],tol=1e-10,m=100):
    n = A.shape[0]
    if len(x0) == 0:
        x = np.random.random(n)
    else:
        x = x0

    xOld = np.zeros(n)
    s=0   
    
    while np.linalg.norm(x-xOld)>tol and s<=m:
        xOld=x.copy()
        for i in range(n):         #Observar que debemos usar las componentes de x anteriores ya obtenidas 
                x[i]=(b[i] - (A[i,:i]@x[:i] + A[i,i+1:]@xOld[i+1:])) / A[i,i] 
        s=s+1
        if s==m:
            print('ATENCIÓN: se alcanzó el número máximo de iteraciones')
    return x,s-1

In [ ]:
Jacobi_ec(A,b) #verifiquemos

In [ ]:
Gauss_Seidel_ec(A,b) #verifiquemos

##  Método SOR

(**Ejercicio 11 de la práctica 5**)

#### a)
Verificar que el sistema $Ax = b$ es equivalente al sistema 
$$(D+\omega L) x=((1-\omega)D - \omega U) x +\omega b,$$
para $\omega \neq 0$. 

#### b)
Escribir el método SOR en la forma $x^{(k+1)} = T_{sor} x^{(k)} + c$ y verificar que la matriz del método es 
$$T(\omega)=(D+\omega L)^{-1}((1-\omega)D - \omega U).$$ 

Calcular el determinante de $T(\omega)$ y probar que si el método converge, debe ser $w \in (0, 2)$.


#### c)
Escribir el programa **SOR** que reciba una matriz $A$, un vector $b$ y un valor $\omega$ y aplique el método de forma matricial.

(Observar que si $\omega=1$ el método ya es conocido ¿Cuál es? Aprovechar ese hecho para testear el código).

Comparar para $\omega=\frac{1}{2}$, $\omega=\frac{2}{3}$ y $\omega=\frac{3}{2}$.

In [ ]:
#####COMPLETAR######

#### d) 
Sabiendo que la coordenada iésima de la solución tiene ecuación
$${\displaystyle x_{i}^{(k+1)}=(1-\omega)x_i^k+{\frac {\omega}{a_{ii}}}\left(-\sum _{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum _{j=i+1}^{n}a_{ij}x_{j}^{(k)}+b_i\right),\quad i=1,2,\dots ,n.}$$
escribir el programa **SOR_ec**.

In [ ]:
#####COMPLETAR######